In [2]:
from config import *

import gensim

import glob
import os
import re

import numpy as np
import pandas as pd

import json

In [3]:
years = [str(year) for year in range(START_YEAR, END_YEAR + 1)]

In [11]:
THEME = 'GDP'
TOKENIZED_ARTICLES_PATH = TEMP_PATH + '/%s/%s_Articles_Tokenized_%s.json'

all_tok_articles = []
for year in years:
    with open(TOKENIZED_ARTICLES_PATH % (THEME, THEME, year)) as f:
        all_tok_articles.extend(json.load(f))
    print(TOKENIZED_ARTICLES_PATH % (THEME, THEME, year), 'done!')

# Only from 1996 - 2016
print(len(all_tok_articles))

# dictionary_all = gensim.corpora.Dictionary(all_tok_articles)
# print('Dictionary created!')

dictionary_all = gensim.corpora.Dictionary.load(TEMP_PATH + '/%s/%s.dict' % (THEME, THEME))
corpus_all = [dictionary_all.doc2bow(doc) for doc in all_tok_articles]


class MyCorpus:
    def __iter__(self):
        for doc in all_tok_articles:
            # assume there's one document per line, tokens separated by whitespace
            yield dictionary_all.doc2bow(doc)

corpus_memory_friendly = MyCorpus()  # doesn't load the corpus into memory!
print(corpus_memory_friendly)

In [13]:
gensim.corpora.MmCorpus.serialize(TEMP_PATH + '/%s/%s.mm' % (THEME, THEME), corpus_memory_friendly)

In [12]:
# Storing the dict/ Corpus for future use incase of issues/ notebook crashes
dictionary_all.save(TEMP_PATH + '/%s/%s.dict' % (THEME, THEME))
gensim.corpora.MmCorpus.serialize(TEMP_PATH + '/%s/%s.mm' % (THEME, THEME), corpus_all)

In [ ]:
del all_tok_articles

In [ ]:
# Get Time Slices

In [14]:
time_slices = pd.read_csv('Summary Stat Tables/%s_Article_Count.csv' % THEME, index_col=0)

In [ ]:
time_slices.sort_index(inplace=True)

In [20]:
time_slices.index = pd.to_datetime(time_slices.index)

time_slices.groupby(time_slices.index.year)['No. of GDP Articles'].sum()[:-4]

1996     30085
1997     21451
1998    121502
1999     67258
2000     53084
2001     57614
2002     54160
2003    102461
2004     91690
2005     97347
2006    171795
2007    185017
2008    219047
2009    200514
2010    102891
2011    117472
2012     97383
2013    113546
2014    133759
2015    167170
2016    210787
Name: No. of GDP Articles, dtype: int64

In [ ]:
time_slices.groupby(time_slices.index.year)['No. of Volatility Articles'].sum().values

In [ ]:
time_slices = time_slices.sort_index()['No. of Volatility Articles'].values

In [ ]:
time_slices

In [ ]:
# LDA Seq Model

In [ ]:
from gensim.models import ldaseqmodel


In [ ]:
time_slices = time_slices.groupby(time_slices.index.year)['No. of Volatility Articles'].sum().values

In [ ]:
ldaseq = ldaseqmodel.LdaSeqModel(corpus=corpus_all, id2word=dictionary_all, time_slice=time_slices, num_topics=10)

In [9]:
import sys

local_vars = list(locals().items())
total_mem = 0
for var, obj in local_vars:
    total_mem += sys.getsizeof(obj)/float(1024*1024)

In [10]:
total_mem

20.745285034179688

In [ ]:
import time


In [ ]:
tic = time.time()

In [ ]:
time.time() - tic

In [ ]:
len(time_slices)

In [24]:
    article_count = pd.read_csv('Summary Stat Tables/%s_Article_Count.csv' % THEME, index_col=0)
    article_count.index = pd.to_datetime(article_count.index)
    article_count = article_count[article_count.index.year <= 2016].sort_index()

In [25]:
article_count

,No. of GDP Articles
1996-01-01,2422
1996-02-01,2553
1996-03-01,2582
1996-04-01,2281
1996-05-01,2605
...,...
2016-08-01,18385
2016-09-01,18363
2016-10-01,18295
2016-11-01,16948
